<a href="https://colab.research.google.com/github/rastringer/promptcraft_notebooks/blob/main/langchain_1hr_blitz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LangChain 1-hr blitz

Models, prompt templates and parsers

In [205]:
! pip install --upgrade google-cloud-aiplatform
! pip install shapely<2.0.0
! pip install langchain langchain-experimental
! pip install pypdf
! pip install pydantic==1.10.8
# Open source vector store
! pip install chromadb==0.3.26
! pip install langchain[docarray]
! pip install typing-inspect==0.8.0 typing_extensions==4.5.0
# For dense vector representations of text
! pip install sentence-transformers

/bin/bash: line 1: 2.0.0: No such file or directory
  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
ERROR: pip's

In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

If you're on Colab, authenticate via the following cell

In [3]:
from google.colab import auth

auth.authenticate_user()

Add your project id and the region

In [4]:
PROJECT_ID = "notebooks-370010"
REGION = "us-central1"

In [5]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

In [6]:
# Utils
import time
from typing import List

# Langchain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI
from google.cloud import aiplatform
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import HumanMessage, SystemMessage

print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.0.331
Vertex AI SDK version: 1.36.0


In [39]:
# We will use chat for some cells
chat = ChatVertexAI(
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True)

# And we will use the general llm for others
llm = VertexAI(
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True)

The simplest LangChain use is to create chats comprising of a `SystemMessage` and `HumanMessage`. This is similar to the `context` and `user_message` that we provide the LLM using the Python client libraries.

In [ ]:
chat([HumanMessage(content="Hello")])

In [8]:
res = chat(
    [
        SystemMessage(
            content="You are an expert chef that thinks of imaginative recipies when people give you ingredients."
        ),
        HumanMessage(content="I have some kidney beans and tomatoes, what would be an easy lunch?"),
    ]
)

print(res.content)

 Here is a recipe for Kidney Bean and Tomato Stew:

Ingredients:

1 tablespoon olive oil
1 onion, chopped
2 cloves garlic, minced
1 (15 ounce) can black beans, rinsed and drained
1 (15 ounce) can diced tomatoes, undrained
1 (10 ounce) can diced tomatoes with green chilies, undrained
1 teaspoon chili powder
1/2 teaspoon ground cumin
1/4 teaspoon salt
1/4 teaspoon black pepper
1/4 cup chopped fresh cilantro

Instructions:

Heat the olive oil in a large saucepan over medium heat. Add the onion and cook until softened, about 5 minutes. Add the garlic and cook for 1 minute more.
Stir in the black beans, tomatoes, diced tomatoes with green chilies, chili powder, cumin, salt, and black pepper. Bring to a boil, then reduce heat and simmer for 15 minutes, or until the beans are heated through.
Stir in the cilantro and serve.


## Prompt templates

LangChain's abstractions such as prompt templates can help keep prompts modular and reusable, especially in large applications which may require long and varied prompts.

Templates may include few-shot examples, instructions, or context.

In [46]:
# The template_string parameters sets the context for the ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [47]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

The chat prompts are envisioned as a series of messages. Notice the `.messages` and `format_messages` methods in the following cells.

In [50]:
# Print out the template
prompt_template.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'))]

In [49]:
#Let's check just the prompt
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [51]:
# Helpful method to keep track of a template's inputs
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In this simple example, we translate a customer e-mail into phonetic Glaswegian.

In [52]:
customer_style = """A polite assistant that writes in phonetic Glaswegian.
"""

In [53]:
customer_email = """
I'm writing this review to express my complete dismay \
and utter horror at the downright disastrous \
coffee maker I purchased from your store. \
It is not at all what I expected. It's a total insult \
to the divine elixir that is coffee!
"""

In [56]:
# The format_messages method sets up the task specified in the template
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [57]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [60]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)
print(customer_response.content)

 A'm writin this review tae express ma complete dismay n utter horror at the downright disastrous coffee maker A bocht fae yer store. It's no ava whit A expectit. It's a total insult tae the divine elixir that is coffee!


## Parsing outputs

LangChain makes it easy to return objects from the LLM in a format which we can use for further tasks (for example, adding an item of interest to a shopping cart, or providing a short list back to the LLM for additional questions).

Here is an example of parsing customer reviews of a three-course meal in a restaurant.

In [86]:
customer_review = """\
The excellent barbecue cauliflower starter left \
a lasting impression -- gorgeous presentation and flavors, really geared the tastebuds into action. \
Moving on to the main course, pretty great also. \
Delicious and flavorful chickpea and vegetable curry. They really nailed the buttery consistency, \
depth and balance of the spices. \
The dessert was a bit bland. I opted for a vegan chocolate mousse, \
hoping for a decadent and indulgent finale to my meal. \
It was very visually appealing but was missing the smooth, velvety \
texture of a great mousse.
"""

review_template = """\
For the input text, extract the following details: \
starter: How did the reviewer find the first course? \
Rate either Poor, Good, or Excellent. \
Do the same for the main course and dessert

Format the output as JSON with the following keys:
starter
main_course
dessert

text: {text}
"""



In [87]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the input text, extract the following details: starter: How did the reviewer find the first course? Rate either Poor, Good, or Excellent. Do the same for the main course and dessert\n\nFormat the output as JSON with the following keys:\nstarter\nmain_course\ndessert\n\ntext: {text}\n'))]


In [88]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages, temperature=0.1)
print(response.content)

 ```JSON
{
  "starter": "Excellent",
  "main_course": "Good",
  "dessert": "Poor"
}
```


Though it looks like JSON, our output is actually a string type.

In [90]:
type(response.content)

str

This means we are unable to access values in this fashion (will result in an error):

In [91]:
response.content.get("main_course")

AttributeError: ignored

This is where Langchain's parser comes in. Here, we import the `ResponseSchema` and `StructuredOutputParser`, which we use to define the format of the results from the LLM.

In [92]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

starter_schema = ResponseSchema(name="starter", description="Review of the starter")
main_course_schema = ResponseSchema(name="main_course", description="Review of the main course")
dessert_schema = ResponseSchema(name="dessert", description="Review of the dessert")

response_schemas = [starter_schema, main_course_schema, dessert_schema]

In [93]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [94]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"starter": string  // Review of the starter
	"main_course": string  // Review of the main course
	"dessert": string  // Review of the dessert
}
```


Now we can update our prior review template to include the format instructions

In [81]:
review_template_2 = """\
For the input text, extract the following details: \
starter: How did the reviewer find the first course? \
Rate either Poor, Good, or Excellent. \
Do the same for the main course and dessert

starter
main_course
dessert

text: {text}

{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [82]:
print(messages[0].content)

For the input text, extract the following details: starter: How did the reviewer find the first course? Rate either Poor, Good, or Excellent. Do the same for the main course and dessert

starter
main_course
dessert

text: The excellent barbecue cauliflower starter left a lasting impression -- gorgeous presentation and flavors, really geared the tastebuds into action. Moving on to the main course, pretty great also. Delicious and flavorful chickpea and vegetable curry. They really nailed the buttery consistency, depth and balance of the spices. The dessert was a bit bland. I opted for a vegan chocolate mousse, hoping for a decadent and indulgent finale to my meal. It was very visually appealing but was missing the smooth, velvety texture of a great mousse.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"starter": string  // Review of the starter
	"main_course": string  // Review of the

In [83]:
response = chat(messages)

Let's try it on the same review

Our response starts as an `AIMessage`

In [84]:
type(response)

langchain.schema.messages.AIMessage

In [85]:
output_dict = output_parser.parse(response.content)
output_dict

{'starter': 'Excellent', 'main_course': 'Good', 'dessert': 'Poor'}

Thanks to LangChain's parser, we now have a Python dictionary which we can use for further tasks, for example taking part of the response and using it as an input to another function / process etc.

In [34]:
type(output_dict)

dict

In [35]:
output_dict.get("main_course")

'Good'

## API chains

Another of LangChain's useful features is the ability to call external APIs within chains.

In this example, we use the `open-meteo.com` API to get weather reports.

In [18]:
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs

llm = VertexAI(temperature=0)
chain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)
chain.run(
    "How is the weather today in Edinburgh, Scotland, in Celsius?"
    )



> Entering new APIChain chain...
 https://api.open-meteo.com/v1/forecast?latitude=55.9533&longitude=-3.1883&hourly=temperature_2m,weathercode&current_weather=true&temperature_unit=celsius
{"latitude":55.96,"longitude":-3.18,"generationtime_ms":0.11491775512695312,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":69.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°C","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2023-11-07T08:45","interval":900,"temperature":7.2,"windspeed":12.1,"winddirection":253,"is_day":1,"weathercode":2},"hourly_units":{"time":"iso8601","temperature_2m":"°C","weathercode":"wmo code"},"hourly":{"time":["2023-11-07T00:00","2023-11-07T01:00","2023-11-07T02:00","2023-11-07T03:00","2023-11-07T04:00","2023-11-07T05:00","2023-11-07T06:00","2023-11-07T07:00","2023-11-07T08:00","2023-11-07T09:00","2023-11-07T10:00","2023-11-07T11:00","2023-11-07T12:00"

' The current temperature in Edinburgh, Scotland is 7.2 °C. The weather is cloudy with a 0% chance of precipitation. There is a 0% chance of sunshine today.'

### Wikipedia

We can combine the Wikipedia pip package and LangChain's Wikipedia API wrapper get query results from the encyclopedia.

In [19]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=82cbed1b89192a81a22f3acb46afa822de8d929e0cdf691eea8fa1197f1ba767
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [22]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikipedia.run("Who was Errol Flynn?")

"Page: Errol Flynn\nSummary: Errol Leslie Thomson Flynn (20 June 1909 – 14 October 1959) was an Australian actor who achieved worldwide fame during the Golden Age of Hollywood. He was known for his romantic swashbuckler roles, frequent partnerships with Olivia de Havilland, and reputation for his womanising and hedonistic personal life. His most notable roles include Robin Hood in The Adventures of Robin Hood (1938), which was later named by the American Film Institute as the 18th greatest hero in American film history, the lead role in Captain Blood (1935), Major Geoffrey Vickers in The Charge of the Light Brigade (1936), and the hero in a number of Westerns such as Dodge City (1939), Santa Fe Trail (1940), and San Antonio (1945).\n\n\n\nPage: Errol Flynn filmography\nSummary: The film appearances of movie actor Errol Flynn (1909–1959) are listed here, including his short films and one unfinished feature.\n\nPage: Sean Flynn (photojournalist)\nSummary: Sean Leslie Flynn (May 31, 1941 

### Google search

In [42]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMRequestsChain, LLMChain

template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)


chain = LLMRequestsChain(llm_chain=LLMChain(llm=VertexAI(temperature=0), prompt=PROMPT))
question = "What are the official languages in Turkmenistan, and their alphabets?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}
chain(inputs)

{'query': 'What are the official languages in Turkmenistan, and their alphabets?',
 'url': 'https://www.google.com/search?q=What+are+the+official+languages+in+Turkmenistan,+and+their+alphabets?',
 'output': ' The official languages in Turkmenistan are Turkmen and Russian. Turkmen is written in a modified Latin alphabet, while Russian is written in the Cyrillic alphabet.'}

## Memory

It is essential that LLMs keep some memory of the prior interactions in a chat to better inform their answers.

LangChain offers several approaches and features in this regard. For all details, see the [Memory](https://python.langchain.com/docs/modules/memory/) section of the documentation.

### ConversationBufferWindowMemory

Maintains a list of the interactions of the conversation over time, using the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large

In [103]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferWindowMemory(k=3)

memory.save_context({"input": "Hi"},
                    {"output": "How are you?"})
memory.save_context({"input": "Fine thanks"},
                    {"output": "Great"})

memory.load_memory_variables({})

{'history': 'Human: Hi\nAI: How are you?\nHuman: Fine thanks\nAI: Great'}

### ConversationTokenBufferMemory

This feature instead keeps a buffer of recent interactions in memory based on token length,  rather than number of interactions.

In [ ]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "All alone, she dreams of the stars!"},
                    {"output": "As she should!"})
memory.save_context({"input": "Baking cookies today?"},
                    {"output": "Behold the cookies!"})
memory.save_context({"input": "Chatbots everywhere?"},
                    {"output": "Certainly!"})

In [ ]:
memory.load_memory_variables({})

{'history': 'Human: Baking cookies today?\nAI: Behold the cookies!\nHuman: Chatbots everywhere?\nAI: Certainly!'}

### Conversation summaries

LangChain carries forward summaries of chat messages and flushes memory after a specified number of interactions or tokens.

Let's first look at using the former, `ConversationBufferWindowMemory`.

We set `verbose=True` to show the prompts and information carried forward by the LLM.

In [104]:
from langchain.memory import ConversationBufferWindowMemory

conversation_with_summary = ConversationChain(
    llm=VertexAI(temperature=0),
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)
conversation_with_summary.predict(input="My favourite sport is fencing. Any tips for how I can go pro?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My favourite sport is fencing. Any tips for how I can go pro?
AI:

> Finished chain.


' Here are some tips on how to become a professional fencer:\n\n1. Start young. The earlier you start fencing, the more time you will have to develop your skills and reach your full potential.\n2. Find a good coach. A good coach can help you develop the proper technique and skills, and can also help you prepare for competitions.\n3. Practice regularly. The more you practice, the better you will become. Try to practice at least three times per week, and make sure to focus on all aspects of fencing, including footwork, bladework, and tactics.\n4. Compete in tournaments. Competition'

In [105]:
conversation_with_summary.predict(input="What equipment do I need?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favourite sport is fencing. Any tips for how I can go pro?
AI:  Here are some tips on how to become a professional fencer:

1. Start young. The earlier you start fencing, the more time you will have to develop your skills and reach your full potential.
2. Find a good coach. A good coach can help you develop the proper technique and skills, and can also help you prepare for competitions.
3. Practice regularly. The more you practice, the better you will become. Try to practice at least three times per week, and make sure to focus on all aspects of fencing, including footwork, bladework, and tactics.
4. Compete in tournaments. Competition
Human: What equipment d

" Here is a list of the equipment you need for fencing:\n\n* A foil, epee, or sabre. The type of weapon you need will depend on the type of fencing you want to do.\n* A mask. This is to protect your face from being hit by the opponent's weapon.\n* A jacket. This is to protect your torso from being hit by the opponent's weapon.\n* A glove. This is to protect your hand from being hit by the opponent's weapon.\n* A pair of breeches. These are special pants that are designed for fencing.\n* A pair of fencing shoes"

In [106]:
conversation_with_summary.predict(input="Who are the greats of the sport I can emulate?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favourite sport is fencing. Any tips for how I can go pro?
AI:  Here are some tips on how to become a professional fencer:

1. Start young. The earlier you start fencing, the more time you will have to develop your skills and reach your full potential.
2. Find a good coach. A good coach can help you develop the proper technique and skills, and can also help you prepare for competitions.
3. Practice regularly. The more you practice, the better you will become. Try to practice at least three times per week, and make sure to focus on all aspects of fencing, including footwork, bladework, and tactics.
4. Compete in tournaments. Competition
Human: What equipment d

" Here are some of the greatest fencers of all time:\n\n* Men's foil:\n    * Stefano Cerioni (Italy)\n    * Alexander Romankov (Russia)\n    * Benjamin Kleibrink (Germany)\n* Women's foil:\n    * Valentina Vezzali (Italy)\n    * Yelena Belova (Russia)\n    * Nam Hyun-hee (South Korea)\n* Men's épée:\n    * Éric Srecki (France)\n    * Pavel Kolobkov (Russia)\n    * Rubén Limardo (Venezuela)\n* Women'"

In [107]:
# Since we have now passed k=2, the LLM will be unable to answer
conversation_with_summary.predict(input="What is my favourite sport?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What equipment do I need?
AI:  Here is a list of the equipment you need for fencing:

* A foil, epee, or sabre. The type of weapon you need will depend on the type of fencing you want to do.
* A mask. This is to protect your face from being hit by the opponent's weapon.
* A jacket. This is to protect your torso from being hit by the opponent's weapon.
* A glove. This is to protect your hand from being hit by the opponent's weapon.
* A pair of breeches. These are special pants that are designed for fencing.
* A pair of fencing shoes
Human: Who are the greats of the sport I can emulate?
AI:  Here are some of the greatest fencers of all time:

* Men's foil:
    * S

' I cannot answer that question as I do not have access to your personal information. \n'

### ConversationTokenBufferMemory

Keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

In [108]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "All alone, she dreams of the stars!"},
                    {"output": "As she should!"})
memory.save_context({"input": "Baking cookies today?"},
                    {"output": "Behold the cookies!"})
memory.save_context({"input": "Chatbots everywhere?"},
                    {"output": "Certainly!"})

In [109]:
memory.load_memory_variables({})

{'history': 'Human: Baking cookies today?\nAI: Behold the cookies!\nHuman: Chatbots everywhere?\nAI: Certainly!'}

### ConversationSummaryBufferMemory

Ensures conversational memory up to a specified token length

In [124]:
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationTokenBufferMemory(llm=llm, max_token_limit=600),
    verbose=True,
)
conversation_with_summary.predict(input="Hi, how are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, how are you?
AI:

> Finished chain.


' I am an AI assistant incapable of experiencing feelings in the same way humans might.  I am, however,  able to assist you with a variety of writing activities. Is there anything I can help you with today?\n'

In [125]:
conversation_with_summary.predict(input="I'm learning the Rust programming language")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  I am an AI assistant incapable of experiencing feelings in the same way humans might.  I am, however,  able to assist you with a variety of writing activities. Is there anything I can help you with today?

Human: I'm learning the Rust programming language
AI:

> Finished chain.


" Rust is a modern systems programming language designed with safety in mind. It provides memory safety without garbage collection, and has a rich type system that prevents many common errors. Rust is also fast and efficient, making it a good choice for performance-critical applications.\nHere are some of the benefits of using Rust:\n\n* Safety: Rust's borrow checker and ownership system prevent many common errors, such as dangling pointers and data races.\n* Speed: Rust is a fast and efficient language, making it a good choice for performance-critical applications.\n* Efficiency: Rust's memory management is very efficient, and it does not require a garbage collector.\n* Versatility: Rust can be used for a wide variety of applications, from operating systems to web browsers.\n\nIf you're interested in learning more about Rust, here are some resources:\n\n* [The Rust Programming Language](https://doc.rust-lang.org/book/)\n* [Rust by Example](https://doc.rust-lang.org/rust-by-example/)\n

In [126]:
conversation_with_summary.predict(input="What's the best book to help me?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What's the best book to help me?
AI:

> Finished chain.


' What topic would you like to learn more about? '

In [127]:
# Notice the buffer here is updated and clears the earlier exchanges
conversation_with_summary.predict(input="Wish me luck!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's the best book to help me?
AI:  What topic would you like to learn more about? 
Human: Wish me luck!
AI:

> Finished chain.


' Good luck!'

The following cell should generate a reply that is clearly generic and missing the previous context of someone trying to learn Rust.

Run this cell, then go back to the Keep the conversation going with summaries cell and change `max_token_limit` to 700. Then re-run the entire conversation and notice how the model relates its ouptut about learning Haskell to the context of someone trying to learn Rust.

In [128]:
conversation_with_summary.predict(input="Would knowing Haskell help me?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's the best book to help me?
AI:  What topic would you like to learn more about? 
Human: Wish me luck!
AI:  Good luck!
Human: Would knowing Haskell help me?
AI:

> Finished chain.


' Haskell is a purely functional programming language. It is known for its conciseness,安全性, and expressive power. It is often used for academic research and for developing high-performance software.\n\nIf you are interested in learning a purely functional programming language, then Haskell is a good choice. It is a well-designed language with a strong community. There are many resources available to help you learn Haskell, including books, tutorials, and online courses.\n\nHere are some of the benefits of learning Haskell:\n\n* It can help you to develop a deeper understanding of programming concepts.\n* It can help you to write more concise and expressive code.\n* It can help you to develop more efficient algorithms.\n* It can help you to write more secure code.\n\nIf you are interested in learning Haskell, I recommend starting with the following resources:\n\n* [Learn You a Haskell for Great Good](https://learnyouahaskell.com/)\n* [Real World Haskell](https://book.realworldhaskell.or

## Chains

Complex applications will require chaining LLMs together, or with other components.

We will cover the following types of chains:

**Sequential chains**

**Router chains**

In [151]:
from langchain.chains import LLMChain

### LLMChain

An LLMChain simply provides a prompt to the LLM.

In [152]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [153]:
chain = LLMChain(llm=llm, prompt=prompt)
product = "A saw for laminate wood"
chain.run(product)

' Assistant: Some potential names for a company that makes a saw for laminate wood include:\n\n- Laminate Saw Solutions\n- The Laminate Saw Company\n- Precision Laminate Saws\n- Laminate Cutting Systems\n- The Laminate Saw Experts\n- Laminate Saw Technology\n- Advanced Laminate Saws\n- Laminate Saw Innovations\n- Laminate Saw Manufacturing\n- Laminate Saw Distribution'

### Sequential chain

A sequential chain makes a series of calls to an LLM. It enables a pipeline-style workflow in which the output from one call becomes the input to the next.

The two types include:

* `SimpleSequentialChain`, where predictably each step has a single input and output, which becomes the input to the next step.

* `SequentialChain`, which allows for multiple inputs and outputs.

In [154]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate

In [155]:
# This is an LLMChain to write a pitch for a new product
# Let's increase the temperature to allow some imagination

llm = VertexAI(temperature=0.7)
template = """You are an entrepreneur. Think of a ground breaking new product and write a short pitch.

Title: {title}
Entrepreneur: This is a pitch for the above product:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
pitch_chain = LLMChain(llm=llm, prompt=prompt_template)

In [156]:
template = """You are a panelist on Dragon's Den. Given a \
description of the product, you are to explain why you think it will \
succeed or fail in the market.

Product pitch: {pitch}
Review by Dragon's Den panelist:"""
prompt_template = PromptTemplate(input_variables=["pitch"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [157]:
# This is the overall chain where we run these two chains in sequence.
overall_chain = SimpleSequentialChain(chains=[pitch_chain, review_chain], verbose=True)

In [158]:
review = overall_chain.run("Portable iced coffee maker")



> Entering new SimpleSequentialChain chain...
 The Iced Coffee Maker is a revolutionary new product that makes iced coffee in just minutes. With its sleek design and compact size, it's perfect for home, office, or on the go. Simply add your favorite coffee grounds and water, and the Iced Coffee Maker will do the rest, brewing and chilling your coffee in no time. Enjoy delicious, refreshing iced coffee whenever you want with the Iced Coffee Maker.
 I think the Iced Coffee Maker is a great product with a lot of potential. Iced coffee is a popular drink, and this product makes it easy to make at home. The Iced Coffee Maker is also very convenient, with a sleek design and compact size. I think this product would be a big hit with consumers, and I would definitely invest in it.

> Finished chain.


### Router chain

A `RouterChain` dynamically selects the next chain to use for a given input.
This feature uses the `MultiPromptChain` to select then answer with the best-suited prompt to the question.

In [160]:
from langchain.chains.router import MultiPromptChain

korean_template = """
You are an expert in korean history and culture.
Here is a question:
{input}
"""

spanish_template = """
You are an expert in spanish history and culture.
Here is a question:
{input}
"""

chinese_template = """
You are an expert in Chinese history and culture.
Here is a question:
{input}
"""

In [161]:
prompt_infos = [
    {
        "name": "korean",
        "description": "Good for answering questions about Korean history and culture",
        "prompt_template": korean_template,
    },
    {
        "name": "spanish",
        "description": "Good for answering questions about Spanish history and culture",
        "prompt_template": spanish_template,
    },
     {
        "name": "chinese",
        "description": "Good for answering questions about Chinese history and culture",
        "prompt_template": chinese_template,
    },
]

In [162]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

llm = VertexAI(temperature=0)

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [163]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [164]:
# Thanks to Deeplearning.ai for this template and for the
# Langchain short course at deeplearning.ai/short-courses/.

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [165]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [166]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

Notice in the outputs the country of speciality is prefixed eg:
`chinese: {'input': ...`, denoting the routing to the correct expert.

In [167]:
chain.run("What was the Han Dynasty?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


chinese: {'input': 'What was the Han Dynasty?'}
> Finished chain.


' Model: \nThe Han Dynasty (206 BC – 220 AD) was the second imperial dynasty of China. It was preceded by the Qin Dynasty and succeeded by the Three Kingdoms period. The Han Dynasty is considered one of the golden ages of Chinese history. During this time, China experienced a period of great prosperity, territorial expansion, and cultural flourishing. The Han Dynasty is also notable for being the first dynasty to officially adopt Confucianism as its state ideology.\n\nThe Han Dynasty was founded by Liu Bang, a peasant leader who overthrew the Qin Dynasty. Liu Bang was a charismatic and capable leader who united China under'

In [168]:
chain.run("What are some of typical dishes in Catalonia?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


spanish: {'input': 'What are some of typical dishes in Catalonia?'}
> Finished chain.


" Model: \nCatalonia is a region in Spain with a rich culinary tradition. Some of the most typical dishes from Catalonia include:\n\n* Pa amb tomàquet: This is a simple but delicious dish of bread rubbed with tomato and drizzled with olive oil. It is often served with jamón serrano or cheese.\n* Escudella i carn d'olla: This is a hearty soup made with pork, chicken, beef, and vegetables. It is typically served with a side of rice or pasta.\n* Calçots: These are a type of green onion that is grilled and served with a romesco sauce. They"

In [169]:
chain.run("How would I greet a friend's parents in Korean?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


korean: {'input': "How would I greet a friend's parents in Korean?"}
> Finished chain.


" Model Assistant: \n안녕하세요 (annyeonghaseyo) is the most common way to greet someone in Korean. It can be used in both formal and informal settings. When greeting a friend's parents, you would typically use the formal form of 안녕하세요, which is 안녕하십니까 (annyeong hasimnikka). You can also say 안녕하세요 to show respect.\n\nHere are some other ways to greet someone in Korean:\n\n* 안녕 (annyeong): This is a more casual way to say hello. It can be used with friends and family.\n* 잘 지"

In [170]:
chain.run("Summarize Don Quixote in a short paragraph")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


spanish: {'input': 'Summarize Don Quixote in a short paragraph'}
> Finished chain.


' Model:\nDon Quixote is a novel by Miguel de Cervantes Saavedra. It was published in two volumes in 1605 and 1615. The novel tells the story of Alonso Quijano, a retired gentleman who becomes obsessed with chivalric romances and decides to become a knight-errant. He changes his name to Don Quixote de la Mancha and sets out on a series of adventures with his squire, Sancho Panza. The novel is a satire of the chivalric romances of the time, and it also explores the themes of love, friendship, and loyalty.\n'

In [171]:
chain.run("How can I fix a carburetor?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': 'How can I fix a carburetor?'}
> Finished chain.


' Assistant: Here are some steps on how to fix a carburetor:\n\n1. Check the fuel level in the carburetor. If the fuel level is low, add more fuel.\n2. Check the air filter. If the air filter is clogged, replace it.\n3. Check the fuel lines. If the fuel lines are cracked or damaged, replace them.\n4. Check the carburetor for leaks. If there are any leaks, seal them with a carburetor repair kit.\n5. Adjust the carburetor. The carburetor may need to be adjusted to ensure that the engine is running properly.\n6. Test the carburetor. Start the'

## Agents and vectorstores

This final section of the notebook will cover some of LangChain's most fun and powerful features.

Agents have access to tools such as JSON, Wikipedia, Web Search, GitHub or Pandas Dataframes, and can access their capabilities depending on user input.

See [here](https://python.langchain.com/docs/integrations/toolkits/) for a full list of agent toolkits.

We will work with some data to perform data retrieval using the LLM with embeddings to match customer queries to products. This is known as Retrieval Augmentated Generation, or RAG.

We will use the Wayfair [WANDS](https://www.aboutwayfair.com/careers/tech-blog/wayfair-releases-wands-the-largest-and-richest-publicly-available-dataset-for-e-commerce-product-search-relevance) dataset of more than 42,000 products. Here are the steps:

* Download the data into a pandas dataframe and take a smaller 1,000-row sample set

* Merge then generate embeddings for the product titles and descriptions

* Prompt an LLM to retrieve details and relevant documents related to queries.


In [172]:
!wget -q https://raw.githubusercontent.com/wayfair/WANDS/main/dataset/product.csv

In [173]:
import pandas as pd
product_df = pd.read_csv("product.csv", sep='\t')

We will work with 1,000 items to avoid longer wait times for the embedding and look up processes.

In [175]:
product_df.head(3)

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0


In [176]:
# Reduce the df to columns of interest
product_df = product_df.filter(["product_id", "product_name", "product_description", "average_rating"], axis=1)

In [177]:
product_df.head(3)

,product_id,product_name,product_description,average_rating
0,0,solid wood platform bed,"good , deep sleep can be quite difficult to ha...",4.5
1,1,all-clad 7 qt . slow cooker,"create delicious slow-cooked meals , from tend...",2.0
2,2,all-clad electrics 6.5 qt . slow cooker,prepare home-cooked meals on any schedule with...,3.0


### Import and initialize pandas dataframe agent

In [181]:
!pip install langchain-experimental

### Pandas agent

In [184]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

agent = create_pandas_dataframe_agent(VertexAI(temperature=0), product_df, verbose=True)

In [185]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
 Thought: The shape of the dataframe can be used to get the number of rows
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 42994
Thought: The number of rows is 42994
Final Answer: 42994

> Finished chain.


'42994'

In [186]:
agent.run("How many beds are there with a rating of > 4?")



> Entering new AgentExecutor chain...
 Thought: To find the number of beds with a rating > 4, I can use the `df.query()` function to filter the dataframe and then use the `df.shape[0]` attribute to get the number of rows in the filtered dataframe.
Action: python_repl_ast
Action Input: df.query('average_rating > 4').shape[0]
Observation: 28399
Thought: There are 28399 beds with a rating > 4.
Final Answer: 28399

> Finished chain.


'28399'

### CSV agent

We can also work directly on a .csv file

In [191]:
pd.DataFrame.to_csv(product_df, "data.csv")

In [193]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent

agent = create_csv_agent(
    VertexAI(temperature=0),
    "data.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [194]:
agent.run("How many rows are there?")



> Entering new AgentExecutor chain...
 Thought: The shape of the dataframe can be used to get the number of rows
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 42994
Thought: The number of rows is 42994
Final Answer: 42994

> Finished chain.


'42994'

In [195]:
agent.run("Do any products descriptions mention polypropylene pile? Output them as JSON please")



> Entering new AgentExecutor chain...
 Thought: To find the products that mention polypropylene pile in their descriptions, I can use the str.contains() method to search for the substring "polypropylene pile" in the product_description column.
Action: python_repl_ast
Action Input: df[df['product_description'].str.contains('polypropylene pile')].to_json(orient='records')
Observation: ValueError: Cannot mask with non-boolean array containing NA / NaN values
Thought: The error message suggests that there are missing values (NA or NaN) in the product_description column. I can handle the missing values by using the na=False parameter in the str.contains() method.
Action: python_repl_ast
Action Input: df[df['product_description'].str.contains('polypropylene pile', na=False)].to_json(orient='records')
Observation: [{"Unnamed: 0":2561,"product_id":2561,"product_name":"gray rug","product_description":"high density 100 % polypropylene pile soft-touch colorfast yarn ( olefin ) . expertly woven 

'[{"Unnamed: 0":2561,"product_id":2561,"product_name":"gray rug","product_description":"high density 100 % polypropylene pile soft-touch colorfast yarn ( olefin ) . expertly woven on jacquard looms .","average_rating":4.5},{"Unnamed: 0":2606,"product_id":2606,"product_name":"colindale red\\/beige rug","product_description":"colindale'

In [196]:
agent.run("What is the square root of all ratings for product names featuring sofas")



> Entering new AgentExecutor chain...
 Thought: To get the square root of all ratings for product names featuring sofas, we can use the following steps:
1. Filter the dataframe to only include product names featuring sofas
2. Get the ratings for the filtered dataframe
3. Take the square root of the ratings
4. Sum the square roots of the ratings
Action: python_repl_ast
Action Input: 
df[df['product_name'].str.contains('sofa')]['average_rating'].apply(np.sqrt).sum()
Observation: NameError: name 'np' is not defined
Thought: To get the square root of all ratings for product names featuring sofas, we can use the following steps:
1. Filter the dataframe to only include product names featuring sofas
2. Get the ratings for the filtered dataframe
3. Take the square root of the ratings
4. Sum the square roots of the ratings
Action: python_repl_ast
Action Input: 
import numpy as np
df[df['product_name'].str.contains('sofa')]['average_rating'].apply(np.sqrt).sum()
Observation: 1436.1503506184686

'1436.1503506184686'

## Vector stores

We will explore embeddings vectors and vector stores in more detail in the [subsequent notebooks](rastringer.io.github.com/promptcraft). Let's see what's possible by concatenating our `product_title` and `product_description` columns and creating a text file from the result. We can then create embeddings and perform various retrieval and Q&A tasks.

We will use the open source [Chroma](https://docs.trychroma.com/) vector store.

In [197]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [198]:
product_df['text_data'] = product_df['product_name'] + " " + product_df['product_description']

In [199]:
# Save the "text_data" column to a text file
text_file_path = "combined_text_data.txt"
product_df['text_data'].to_csv(text_file_path, sep='\t', index=False, header=False)


In [200]:
# load the document and split it into chunks
loader = TextLoader("combined_text_data.txt")
documents = loader.load()

In [201]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

docs = text_splitter.split_documents(documents)

In [202]:
len(docs)

15742

In [203]:
from langchain.vectorstores import Chroma

# Clear any previous vector store
!rm -rf ./docs/chroma

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embedding_function)

In [ ]:
query = "Is there a slow cooker?"
docs = db.similarity_search(query, n_results=2)

In [ ]:
docs[0]

In [ ]:
query = "Recommend a durable door mat"
docs = db.similarity_search(query, n_results=2)

### Retrieval

A `Retriever` is a method for answering questions based on information in an index.

Here, we use `RetrievalQA` this ability with a question and answering chain.

In [ ]:
from langchain.chains import RetrievalQA

llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever()
)

### Prompt

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. \
If you don't know the answer, just say that you don't know, \
don't try to make up an answer. Use three sentences maximum. \
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)


In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "Can you recommend comfortable bed sheets?"
result = qa_chain({"query": question})
result["result"]